In [1]:
1+1


2

In [1]:
!pip install streamlit


  Using cached altair-6.0.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached pandas-2.3.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.37.0-py3-none-any.whl.metadata (2.8 kB)


[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from db_connection import setup_database
from intent import identify_intent
from retrieve import ask_sql_ai, ask_rag_ai, ask_both_ai
from retrieve import validate_query
from ingest import ingest_to_knowledge_base

def main():
    # 1. Initialize System
    setup_database()
    # ingest_data('info.txt') # Run once
    files_to_process = [
        '../data/all_product_specs.txt', 
        '../data/all_warranties.txt', 
        '../data/delivery_koombiyo.txt'
    ]
    #ingest_to_knowledge_base(files_to_process)
    query = "why order 118 is delay?"
    print(f"📚 User Query : {query}")
    
    # 2. Guardrail Check (Security & Max Token Check)
    is_safe, error_message = validate_query(query)
    if not is_safe:
        print(f"🤖 AI Guardrail Response: {error_message}")
        return  # 🛑 STOP processing immediately

    # 3. Intent Routing
    route = identify_intent(query)
    
    # 4. Execution Pipeline
    if "BOTH" in route:
        print("🔀 Path: BOTH (SQL + RAG)")
        answer = ask_both_ai(query)
    elif "SQL" in route:
        print("🔍 Path: SQL")
        answer = ask_sql_ai(query)
    else:
        print("📚 Path: RAG")
        answer = ask_rag_ai(query)

    print(f"\n🤖 AI Response:\n{answer}")

if __name__ == "__main__":
    main()


this is main now i need to this pos rag to give streamlit interface to ask question 

In [1]:
!pip list | findstr psycopg2

psycopg2-binary         2.9.11


In [4]:
import psycopg2

DB_CONFIG = {
    "dbname": "Pos_dbc",  # Verified from your screenshot
    "user": "postgres",  # Note the spelling in your screenshot: "postgrdes"
    "password": "1111",
    "host": "localhost",
    "port": "5432"
}

conn = psycopg2.connect(**DB_CONFIG)
print("✅ Connected!")
conn.close()

✅ Connected!


In [5]:
import psycopg2
from psycopg2.extras import RealDictCursor
import os

# Ensure the DB_CONFIG exactly matches psycopg2 expectations
DB_CONFIG = {
    "dbname": "Pos_dbc",  # Verified from your screenshot
    "user": "postgres",  # Note the spelling in your screenshot: "postgrdes"
    "password": "1111",
    "host": "localhost",
    "port": "5432"
}

def setup_database():
    try:
        # Use keyword unpacking; Pylance may still warn, but it WILL run correctly.
        # You can add # type: ignore if you want to hide the red underline.
        conn = psycopg2.connect(**DB_CONFIG) # type: ignore
        cur = conn.cursor()
        
        # Enable pgvector and create knowledge base table
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
        
        # Step 2: Create knowledge_base table (MiniLM 384 dimensions)
        cur.execute("""
            CREATE TABLE IF NOT EXISTS knowledge_base (
                kb_id SERIAL PRIMARY KEY,
                title TEXT,
                content TEXT NOT NULL,
                document_type VARCHAR(50),
                source VARCHAR(100),
                embedding vector(384)
            );
        """)
        
        conn.commit()
        print("✅ Database connection successful and schema ready.")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
    finally:
        if 'conn' in locals():
            cur.close()
            conn.close()



setup_database()            

✅ Database connection successful and schema ready.


In [ ]:

import os
import psycopg2
from sentence_transformers import SentenceTransformer
from groq import Groq
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key using the key name defined in the .env file
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if GROQ_API_KEY is not None:
    print(f"API Key successfully loaded. Key starts with: {GROQ_API_KEY[:5]}...")
else:
    print("API Key not found. Please check your .env file and environment setup.")


# --- 1. CONFIGURATION ---
DB_CONFIG = {
    "dbname": "Pos_dbc",
    "user": "postgres",
    "password": "1111",
    "host": "localhost",
    "port": "5432"
}



# Model for local embeddings (384 dimensions)
embed_model = SentenceTransformer('all-MiniLM-L6-v2') 
groq_client = Groq(api_key=GROQ_API_KEY)

# Complete Schema Info for SQL Insights
SCHEMA_INFO = """
The database 'Pos_dbc' has these tables:
1. product (product_id, name, category_id, brand, current_price, policy_id, embedding vector(384))
2. stock (product_id, quantity, last_updated)
3. "order" (order_id, customer_id, staff_id, courier_id, total_price, order_date, status_id)
4. order_item (order_id, product_id, quantity, price_at_sale)
5. knowledge_base (kb_id, title, content, document_type, source, embedding vector(384))
6. courier (courier_id, service_name, contact_number)
7. order_status (status_id, status_name)
8. staff (staff_id, name, role)
9. customer (customer_id, name, phone, email, address)
10. price_change_log (log_id, product_id, previous_price, new_price, change_reason, change_date)

Note: 'order' is a reserved word; always use double quotes: "order".
Note:price mention with LKR
"""




# --- 2. DATABASE SETUP ---
def setup_database():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS knowledge_base (
            kb_id SERIAL PRIMARY KEY,
            content TEXT NOT NULL,
            embedding vector(384)
        );
    """)
    cur.execute("ALTER TABLE product ADD COLUMN IF NOT EXISTS embedding vector(384);")
    conn.commit()
    cur.close()
    conn.close()
    print("✅ Database prepared.")

# --- 3. INGESTION (Process info.txt) ---
def ingest_data(file_path):
    if not os.path.exists(file_path):
        print(f"❌ Error: {file_path} not found.")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. PARAGRAPH-WISE CHUNKING
    # Splits by double newlines to keep specific product info together
    chunks = [c.strip() for c in text.split('\n\n') if c.strip()]
    
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Task A: Populate Knowledge Base
    print(f"📦 Processing {len(chunks)} paragraphs for Knowledge Base...")
    for chunk in chunks:
        vector = embed_model.encode(chunk).tolist()
        cur.execute(
            "INSERT INTO knowledge_base (content, embedding) VALUES (%s, %s)",
            (chunk, vector)
        )

    # Task B: Fill Product Table Embeddings
    print("🔄 Updating product table embeddings...")
    cur.execute("SELECT product_id, name FROM product;")
    products = cur.fetchall()

    for p_id, p_name in products:
        # Find the specific paragraph in info.txt that mentions this product
        matching_paragraph = next((c for c in chunks if p_name in c), None)
        
        if matching_paragraph:
            # Generate embedding for the specific product description
            prod_vector = embed_model.encode(matching_paragraph).tolist()
            cur.execute(
                "UPDATE product SET embedding = %s WHERE product_id = %s",
                (prod_vector, p_id)
            )
            print(f"✅ Embedded: {p_name}")
        else:
            print(f"⚠️ No description found for: {p_name}")

    conn.commit()
    cur.close()
    conn.close()
    print("🚀 Ingestion and Product synchronization complete.")

# --- 4. RAG SEARCH (Vector Search) ---
def ask_rag_ai(question):
    question_vector = embed_model.encode(question).tolist()
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Each subquery is wrapped in () to allow its own ORDER BY and LIMIT
    search_query = """
        (SELECT content FROM knowledge_base
         ORDER BY embedding <=> %s::vector LIMIT 2)
        UNION ALL
        (SELECT name || ': ' || brand FROM product
         ORDER BY embedding <=> %s::vector LIMIT 1);
    """
    
    try:
        cur.execute(search_query, (question_vector, question_vector))
        results = cur.fetchall()
        context = "\n".join([r[0] for r in results])
        
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {
                    "role": "system", 
                    "content": f"You are a strict POS assistant. ONLY use this info: {context}. "
                               "If the answer is not in the info, say you do not have that data."
                },
                {"role": "user", "content": question}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"❌ Retrieval Error: {e}"
    finally:
        cur.close()
        conn.close()

# --- 5. SQL INSIGHTS (Text-to-SQL) ---
def ask_sql_ai(question):
    # Updated Prompt: Forces the AI to use fuzzy matching (ILIKE and %)
    sql_prompt = f"""
    System: You are a Read-Only PostgreSQL generator. 
    Task: Generate a SELECT query to answer: {question}

    SCHEMA:
    {SCHEMA_INFO}

    STRICT RULES (Violating these will break the system):
    1. Respond with ONLY the raw SQL string. 
    2. NO markdown code blocks (no ```sql).
    3. NO conversational text, explanations, or introductory remarks.
    4. Use 'ILIKE' with wildcards (%) for all text searches to ensure fuzzy matching.
    Example: For 'i phone 15 128gb', use ILIKE '%iPhone%15%128GB%'
    5. ONLY 'SELECT' queries are allowed.
    6. FORBIDDEN COMMANDS: DELETE, DROP, TRUNCATE, UPDATE, INSERT, ALTER, CREATE.

    Query:
    """
    
    sql_response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": sql_prompt}]
    )
    
    # Clean the output to ensure it runs in psycopg2
    generated_sql = sql_response.choices[0].message.content.strip()
    generated_sql = generated_sql.replace("```sql", "").replace("```", "").replace("\n", " ")

    print(f"🚀 Optimized SQL: {generated_sql}")

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    try:
        cur.execute(generated_sql)
        db_results = cur.fetchall()
        
        # Let Groq explain the final price to the user
        final_answer = groq_client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[
                {"role": "system", "content": "You are a helpful POS assistant."},
                {"role": "user", "content": f"User asked: {question} \n Database result: {db_results}"}
            ]
        )
        return final_answer.choices[0].message.content
    except Exception as e:
        return f"❌ SQL Error: {e}"
    finally:
        cur.close()
        conn.close()

# --- 6. HYBRID AGENT (The Router) ---
def hybrid_ai_assistant(question):
    routing_prompt = f"Does this question need a SQL query (for sales/stock/numbers) or RAG (for product specs/policies)? Respond with ONLY 'SQL' or 'RAG' . Question: {question}"
    route = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": routing_prompt}]
    ).choices[0].message.content.strip()

    if "SQL" in route:
        print("🔍 Routing to SQL Database...")
        return ask_sql_ai(question)
    else:
        print("📚 Routing to Knowledge Base (RAG)...")
        return ask_rag_ai(question)

# --- EXECUTION ---
if __name__ == "__main__":
    setup_database()
    #ingest_data('info.txt') # Run this once
    
    query = "what is order_stus id of order 118  ?"
    #query = "give prices of I Phone s and give names of it  ?"
    answer = hybrid_ai_assistant(query)
    print(f"\n🤖 AI Response:\n{answer}")

✅ Database prepared.
🔍 Routing to SQL Database...
🚀 Optimized SQL: SELECT T1.status_id FROM order_status AS T1 INNER JOIN "order" AS T2 ON T1.status_id = T2.status_id WHERE T2.order_id = 118;

🤖 AI Response:
The `order_stus` ID of order 118 is **2**. 

Let me explain what I did: 

The database result was `[(2,)]`. This is a tuple within a list. To get the `order_stus` ID, I simply extracted the value within the tuple, which is `2`. 

Is there anything else I can help you with?


In [ ]:

import os
import psycopg2
from sentence_transformers import SentenceTransformer
from groq import Groq

# --- 1. CONFIGURATION ---
DB_CONFIG = {
    "dbname": "Pos_dbc",
    "user": "postgres",
    "password": "1111",
    "host": "localhost",
    "port": "5432"
}

# Replace with your actual Groq API Key


# Model for local embeddings (384 dimensions)
embed_model = SentenceTransformer('all-MiniLM-L6-v2') 
groq_client = Groq(api_key=GROQ_API_KEY)

# Complete Schema Info for SQL Insights
SCHEMA_INFO = """
The database 'Pos_dbc' has these tables:
1. product (product_id, name, category_id, brand, current_price, policy_id, embedding vector(384))
2. stock (product_id, quantity, last_updated)
3. "order" (order_id, customer_id, staff_id, courier_id, total_price, order_date, status_id)
4. order_item (order_id, product_id, quantity, price_at_sale)
5. knowledge_base (kb_id, title, content, document_type, source, embedding vector(384))
6. courier (courier_id, service_name, contact_number)
7. order_status (status_id, status_name)
8. staff (staff_id, name, role)
9. customer (customer_id, name, phone, email, address)
10. price_change_log (log_id, product_id, previous_price, new_price, change_reason, change_date)

Note: 'order' is a reserved word; always use double quotes: "order".
Note:price mention with LKR
"""




# --- 2. DATABASE SETUP ---
def setup_database():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    cur.execute("""
        CREATE TABLE IF NOT EXISTS knowledge_base (
            kb_id SERIAL PRIMARY KEY,
            content TEXT NOT NULL,
            embedding vector(384)
        );
    """)
    cur.execute("ALTER TABLE product ADD COLUMN IF NOT EXISTS embedding vector(384);")
    conn.commit()
    cur.close()
    conn.close()
    print("✅ Database prepared.")

# --- 3. INGESTION (Process info.txt) ---
def ingest_data(file_path):
    if not os.path.exists(file_path):
        print(f"❌ Error: {file_path} not found.")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # 1. PARAGRAPH-WISE CHUNKING
    # Splits by double newlines to keep specific product info together
    chunks = [c.strip() for c in text.split('\n\n') if c.strip()]
    
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Task A: Populate Knowledge Base
    print(f"📦 Processing {len(chunks)} paragraphs for Knowledge Base...")
    for chunk in chunks:
        vector = embed_model.encode(chunk).tolist()
        cur.execute(
            "INSERT INTO knowledge_base (content, embedding) VALUES (%s, %s)",
            (chunk, vector)
        )

    # Task B: Fill Product Table Embeddings
    print("🔄 Updating product table embeddings...")
    cur.execute("SELECT product_id, name FROM product;")
    products = cur.fetchall()

    for p_id, p_name in products:
        # Find the specific paragraph in info.txt that mentions this product
        matching_paragraph = next((c for c in chunks if p_name in c), None)
        
        if matching_paragraph:
            # Generate embedding for the specific product description
            prod_vector = embed_model.encode(matching_paragraph).tolist()
            cur.execute(
                "UPDATE product SET embedding = %s WHERE product_id = %s",
                (prod_vector, p_id)
            )
            print(f"✅ Embedded: {p_name}")
        else:
            print(f"⚠️ No description found for: {p_name}")

    conn.commit()
    cur.close()
    conn.close()
    print("🚀 Ingestion and Product synchronization complete.")



# --- IMPROVED: Context-Aware RAG Search ---
def get_rag_context_with_sql_data(question, sql_results=None):
    """
    Enhanced RAG that uses SQL results to improve vector search
    
    Args:
        question: User's question
        sql_results: Results from SQL query (optional, improves search)
    
    Returns:
        Relevant context from knowledge base
    """
    # Build enhanced search query using SQL results
    if sql_results:
        # Extract key information from SQL results
        enhanced_query = f"{question} {str(sql_results)}"
    else:
        enhanced_query = question
    
    question_vector = embed_model.encode(enhanced_query).tolist()
    
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Improved search: Search knowledge base with better context
    search_query = """
        (SELECT content FROM knowledge_base
         ORDER BY embedding <=> %s::vector LIMIT 2)
        UNION ALL
        (SELECT name || ': ' || brand FROM product
         ORDER BY embedding <=> %s::vector LIMIT 1);
    """
    
    
    cur.execute(search_query, (question_vector, question_vector))
    results = cur.fetchall()
    cur.close()
    conn.close()
    
    # Format results with context
    context_parts = []
    for title, content, doc_type, similarity in results:
        if similarity > 0.6:  # Only include relevant results
            context_parts.append(f"[{doc_type}] {title}: {content[:300]}")
    
    return "\n\n".join(context_parts) if context_parts else "No relevant context found."


# --- IMPROVED: SQL with Better Error Handling ---
def get_sql_data(question):
    """Get data from SQL - returns both raw results and formatted answer"""
    
    sql_prompt = f"""
    System: You are a Read-Only PostgreSQL generator. 
    Task: Generate a SELECT query to answer: {question}

    SCHEMA:
    {SCHEMA_INFO}

    STRICT RULES:
    1. Respond with ONLY the raw SQL string. 
    2. NO markdown code blocks (no ```sql).
    3. NO conversational text, explanations, or introductory remarks.
    4. Use 'ILIKE' with wildcards (%) for all text searches.
    5. ONLY 'SELECT' queries allowed.
    6. For order status questions, JOIN with order_status table to get status_name.
    7. For delay questions, include delay_reason column.

    Query:
    """
    
    sql_response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": sql_prompt}]
    )
    
    generated_sql = sql_response.choices[0].message.content.strip()
    generated_sql = generated_sql.replace("```sql", "").replace("```", "").replace("\n", " ")

    print(f"🚀 Generated SQL: {generated_sql}")

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    try:
        cur.execute(generated_sql)
        db_results = cur.fetchall()
        
        # Return both raw results and formatted version
        return {
            'success': True,
            'raw_results': db_results,
            'sql_query': generated_sql,
            'row_count': len(db_results)
        }
    
    except Exception as e:
        print(f"❌ SQL Error: {e}")
        return {
            'success': False,
            'error': str(e),
            'sql_query': generated_sql
        }
    finally:
        cur.close()
        conn.close()


# --- NEW: Intelligent Hybrid Search (SQL → Vector → Answer) ---
def ask_hybrid_ai_intelligent(question):
    """
    Intelligent hybrid search that uses SQL results to guide vector search
    
    Flow:
    1. Get SQL data first (facts)
    2. Use SQL results to improve vector search (context)
    3. Combine for final answer
    """
    
    print("\n" + "="*60)
    print("🔍 STEP 1: Getting SQL Data...")
    print("="*60)
    
    # Step 1: Get SQL data
    sql_data = get_sql_data(question)
    
    if not sql_data['success']:
        return f"❌ Database error: {sql_data['error']}"
    
    db_results = sql_data['raw_results']
    print(f"✅ Found {sql_data['row_count']} rows")
    print(f"📊 Results: {db_results}")
    
    # Step 2: Check if we need additional context from knowledge base
    needs_context = any(keyword in question.lower() 
                       for keyword in ['why', 'reason', 'explain', 'because', 'how'])
    
    if needs_context:
        print("\n" + "="*60)
        print("🔍 STEP 2: Searching Knowledge Base for Context...")
        print("="*60)
        
        # Use SQL results to enhance vector search
        rag_context = get_rag_context_with_sql_data(question, db_results)
        print(f"✅ Found relevant context")
        
        print("\n" + "="*60)
        print("🤖 STEP 3: Generating Final Answer...")
        print("="*60)
        
        # Step 3: Combine SQL + Vector for final answer
        final_prompt = f"""
        You are a helpful POS assistant. Answer the user's question using BOTH database facts AND knowledge base context.
        
        User Question: {question}
        
        DATABASE FACTS:
        {db_results}
        
        KNOWLEDGE BASE CONTEXT:
        {rag_context}
        
        Instructions:
        1. Start with the direct answer from database
        2. Then add explanation/context from knowledge base
        3. Be concise but complete
        4. If database shows status_id, translate it (1=Processing, 2=Delayed, 4=Complete)
        """
        
        final_answer = groq_client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": final_prompt}]
        )
        
        return final_answer.choices[0].message.content
    
    else:
        # No context needed, just format SQL results
        print("\n" + "="*60)
        print("🤖 STEP 2: Formatting SQL Results...")
        print("="*60)
        
        format_prompt = f"""
        Format this database result in a natural, user-friendly way.
        
        User Question: {question}
        Database Result: {db_results}
        
        Provide a clear, concise answer.
        """
        
        formatted_answer = groq_client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": format_prompt}]
        )
        
        return formatted_answer.choices[0].message.content


# --- IMPROVED: Smarter Router ---
def hybrid_ai_assistant(question):
    """
    Smart routing with better logic
    """
    
    routing_prompt = f"""
    Analyze this question and determine the best approach:
    
    Question: {question}
    
    Choose ONE:
    - SQL: Question asks for specific data, numbers, prices, stock levels, order details
    - RAG: Question asks about policies, procedures, general knowledge, recommendations
    - HYBRID: Question asks for data AND wants explanation/reason (contains 'why', 'reason', 'explain')
    
    Respond with ONLY: SQL, RAG, or HYBRID
    """
    
    route = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": routing_prompt}]
    ).choices[0].message.content.strip().upper()

    print(f"\n🛤️  Route: {route}")
    print("="*60)

    if "HYBRID" in route:
        return ask_hybrid_ai_intelligent(question)
    
    elif "SQL" in route:
        print("📊 Using SQL-only path...")
        sql_data = get_sql_data(question)
        
        if sql_data['success']:
            # Format results nicely
            format_prompt = f"Question: {question}\nDatabase result: {sql_data['raw_results']}\n\nProvide a clear answer."
            
            answer = groq_client.chat.completions.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[{"role": "user", "content": format_prompt}]
            )
            return answer.choices[0].message.content
        else:
            return f"❌ Database error: {sql_data['error']}"
    
    else:  # RAG
        print("📚 Using RAG-only path...")
        context = get_rag_context_with_sql_data(question)
        
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": f"Answer based on this context:\n{context}"},
                {"role": "user", "content": question}
            ]
        )
        return response.choices[0].message.content



if __name__ == "__main__":
    setup_database()
    query = "what is order_status id of order 118 and is it delayed or not if it is delayed then why?"
    answer = hybrid_ai_assistant(query)
    print(f"\n🤖 AI Response:\n{answer}")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ca33746e-aa51-4d8d-a7be-30bbd5527ecb)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: a7037e39-e9c7-4bb1-95a7-d32ce185b16b)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./config_sentence_transformers.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 1d1382f3-32bc-441a-b1f6-e5474bc1cae5)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./config_sentence_transformers.json
Retrying in 2s [Retry 2/5].
'(ReadTimeoutError(

✅ Database prepared.

🛤️  Route: HYBRID

🔍 STEP 1: Getting SQL Data...


KeyboardInterrupt: 

In [46]:
# --- IMPROVED: Context-Aware RAG Search ---
def get_rag_context_with_sql_data(question, sql_results=None):
    """
    Enhanced RAG that uses SQL results to improve vector search
    
    Args:
        question: User's question
        sql_results: Results from SQL query (optional, improves search)
    
    Returns:
        Relevant context from knowledge base
    """
    # Build enhanced search query using SQL results
    if sql_results:
        # Extract key information from SQL results
        enhanced_query = f"{question} {str(sql_results)}"
    else:
        enhanced_query = question
    
    question_vector = embed_model.encode(enhanced_query).tolist()
    
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # Improved search: Search knowledge base with better context
    search_query = """
        (SELECT content FROM knowledge_base
         ORDER BY embedding <=> %s::vector LIMIT 2)
        UNION ALL
        (SELECT name || ': ' || brand FROM product
         ORDER BY embedding <=> %s::vector LIMIT 1);
    """
    
    
    cur.execute(search_query, (question_vector, question_vector))
    results = cur.fetchall()
    cur.close()
    conn.close()
    
    # Format results with context
    context_parts = []
    for title, content, doc_type, similarity in results:
        if similarity > 0.6:  # Only include relevant results
            context_parts.append(f"[{doc_type}] {title}: {content[:300]}")
    
    return "\n\n".join(context_parts) if context_parts else "No relevant context found."


# --- IMPROVED: SQL with Better Error Handling ---
def get_sql_data(question):
    """Get data from SQL - returns both raw results and formatted answer"""
    
    sql_prompt = f"""
    System: You are a Read-Only PostgreSQL generator. 
    Task: Generate a SELECT query to answer: {question}

    SCHEMA:
    {SCHEMA_INFO}

    STRICT RULES:
    1. Respond with ONLY the raw SQL string. 
    2. NO markdown code blocks (no ```sql).
    3. NO conversational text, explanations, or introductory remarks.
    4. Use 'ILIKE' with wildcards (%) for all text searches.
    5. ONLY 'SELECT' queries allowed.
    6. For order status questions, JOIN with order_status table to get status_name.
    7. For delay questions, include delay_reason column.

    Query:
    """
    
    sql_response = groq_client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{"role": "user", "content": sql_prompt}]
    )
    
    generated_sql = sql_response.choices[0].message.content.strip()
    generated_sql = generated_sql.replace("```sql", "").replace("```", "").replace("\n", " ")

    print(f"🚀 Generated SQL: {generated_sql}")

    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    try:
        cur.execute(generated_sql)
        db_results = cur.fetchall()
        
        # Return both raw results and formatted version
        return {
            'success': True,
            'raw_results': db_results,
            'sql_query': generated_sql,
            'row_count': len(db_results)
        }
    
    except Exception as e:
        print(f"❌ SQL Error: {e}")
        return {
            'success': False,
            'error': str(e),
            'sql_query': generated_sql
        }
    finally:
        cur.close()
        conn.close()


# --- NEW: Intelligent Hybrid Search (SQL → Vector → Answer) ---
def ask_hybrid_ai_intelligent(question):
    """
    Intelligent hybrid search that uses SQL results to guide vector search
    
    Flow:
    1. Get SQL data first (facts)
    2. Use SQL results to improve vector search (context)
    3. Combine for final answer
    """
    
    print("\n" + "="*60)
    print("🔍 STEP 1: Getting SQL Data...")
    print("="*60)
    
    # Step 1: Get SQL data
    sql_data = get_sql_data(question)
    
    if not sql_data['success']:
        return f"❌ Database error: {sql_data['error']}"
    
    db_results = sql_data['raw_results']
    print(f"✅ Found {sql_data['row_count']} rows")
    print(f"📊 Results: {db_results}")
    
    # Step 2: Check if we need additional context from knowledge base
    needs_context = any(keyword in question.lower() 
                       for keyword in ['why', 'reason', 'explain', 'because', 'how'])
    
    if needs_context:
        print("\n" + "="*60)
        print("🔍 STEP 2: Searching Knowledge Base for Context...")
        print("="*60)
        
        # Use SQL results to enhance vector search
        rag_context = get_rag_context_with_sql_data(question, db_results)
        print(f"✅ Found relevant context")
        
        print("\n" + "="*60)
        print("🤖 STEP 3: Generating Final Answer...")
        print("="*60)
        
        # Step 3: Combine SQL + Vector for final answer
        final_prompt = f"""
        You are a helpful POS assistant. Answer the user's question using BOTH database facts AND knowledge base context.
        
        User Question: {question}
        
        DATABASE FACTS:
        {db_results}
        
        KNOWLEDGE BASE CONTEXT:
        {rag_context}
        
        Instructions:
        1. Start with the direct answer from database
        2. Then add explanation/context from knowledge base
        3. Be concise but complete
        4. If database shows status_id, translate it (1=Processing, 2=Delayed, 4=Complete)
        """
        
        final_answer = groq_client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": final_prompt}]
        )
        
        return final_answer.choices[0].message.content
    
    else:
        # No context needed, just format SQL results
        print("\n" + "="*60)
        print("🤖 STEP 2: Formatting SQL Results...")
        print("="*60)
        
        format_prompt = f"""
        Format this database result in a natural, user-friendly way.
        
        User Question: {question}
        Database Result: {db_results}
        
        Provide a clear, concise answer.
        """
        
        formatted_answer = groq_client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": format_prompt}]
        )
        
        return formatted_answer.choices[0].message.content


# --- IMPROVED: Smarter Router ---
def hybrid_ai_assistant(question):
    """
    Smart routing with better logic
    """
    
    routing_prompt = f"""
    Analyze this question and determine the best approach:
    
    Question: {question}
    
    Choose ONE:
    - SQL: Question asks for specific data, numbers, prices, stock levels, order details
    - RAG: Question asks about policies, procedures, general knowledge, recommendations
    - HYBRID: Question asks for data AND wants explanation/reason (contains 'why', 'reason', 'explain')
    
    Respond with ONLY: SQL, RAG, or HYBRID
    """
    
    route = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": routing_prompt}]
    ).choices[0].message.content.strip().upper()

    print(f"\n🛤️  Route: {route}")
    print("="*60)

    if "HYBRID" in route:
        return ask_hybrid_ai_intelligent(question)
    
    elif "SQL" in route:
        print("📊 Using SQL-only path...")
        sql_data = get_sql_data(question)
        
        if sql_data['success']:
            # Format results nicely
            format_prompt = f"Question: {question}\nDatabase result: {sql_data['raw_results']}\n\nProvide a clear answer."
            
            answer = groq_client.chat.completions.create(
                model="meta-llama/llama-4-scout-17b-16e-instruct",
                messages=[{"role": "user", "content": format_prompt}]
            )
            return answer.choices[0].message.content
        else:
            return f"❌ Database error: {sql_data['error']}"
    
    else:  # RAG
        print("📚 Using RAG-only path...")
        context = get_rag_context_with_sql_data(question)
        
        response = groq_client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[
                {"role": "system", "content": f"Answer based on this context:\n{context}"},
                {"role": "user", "content": question}
            ]
        )
        return response.choices[0].message.content


# --- EXAMPLE USAGE ---
if __name__ == "__main__":
    setup_database()
    
    # Test queries
    test_queries = [
        "what is order_status id of order 118 and is it delayed or not if it is delayed then why?",
        "what is the price of iPhone 15 Pro?",
        "why are deliveries delayed?",
        "show me all delayed orders and explain why"
    ]
    
    for query in test_queries:
        print("\n" + "="*80)
        print(f"❓ QUESTION: {query}")
        print("="*80)
        
        answer = hybrid_ai_assistant(query)
        
        print("\n" + "="*80)
        print("🤖 ANSWER:")
        print("="*80)
        print(answer)
        print("\n")

✅ Database prepared.

❓ QUESTION: what is order_status id of order 118 and is it delayed or not if it is delayed then why?

🛤️  Route: HYBRID

🔍 STEP 1: Getting SQL Data...
🚀 Generated SQL: SELECT T1.status_id, T2.status_name, T1.delay_reason  FROM "order" AS T1  JOIN order_status AS T2 ON T1.status_id = T2.status_id  LEFT JOIN (     SELECT order_id, status_id AS delay_reason      FROM order_status_history      WHERE status_id  = 5 ) AS T3 ON T1.order_id = T3.order_id  WHERE T1.order_id = 118;
❌ SQL Error: relation "order_status_history" does not exist
LINE 1: ...ECT order_id, status_id AS delay_reason      FROM order_stat...
                                                             ^


🤖 ANSWER:
❌ Database error: relation "order_status_history" does not exist
LINE 1: ...ECT order_id, status_id AS delay_reason      FROM order_stat...
                                                             ^




❓ QUESTION: what is the price of iPhone 15 Pro?

🛤️  Route: HYBRID

(EXPLANATION: TH

ValueError: not enough values to unpack (expected 4, got 1)